In [1]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk
import math
from spyci import spyci

#### Technology data

In [2]:
n = lk('../../lookuptable/skywater130a/nfet_01v8.mat')
p = lk('../../lookuptable/skywater130a/pfet_01v8.mat')


#### Specifications

In [43]:
# define the given parameters as taken from the specification table or initial guesses
c_load = 1e-12
gm_id = [5, 15, 12, 5, 15, 12, 5]
l = [5, 5, 5, 5, 5, 5, 5]
gm = np.zeros(len(gm_id))
id = np.zeros(len(gm_id))
gm_gds = np.zeros(len(gm_id))
gds = np.zeros(len(gm_id))
f_bw = 1e6 # -3dB bandwidth of the voltage buffer
i_total_limit = 10e-6
i_bias_in = 20e-6
output_voltage = 1.6
vin_min = 0.8
vin_max = 0.9
vdd_min = 1.75
vdd_max = 1.85
vds_headroom = 0.75

In [ ]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buffer specification
# note that the -3dB bandwidth of the voltage buffer with gain Av=1 is equal to the unity gain bandwidth
# of the ota, hence we wet them equal here
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loading
Cc = c_load * 2.6
gm[1] = 2 * np.pi * f_bw * Cc
fp2 = 3 * f_bw
gm[4] = 2 * np.pi * fp2 * c_load
# gm[4] = 10* gm[1]
Rc = 1/(gm[4])*c_load/Cc
print(f"gm[1] = {round(gm[1]/1e-6,4)} uS | gm[4] = {round(gm[4]/1e-6,4)} uS")
print(f"Rc = {round(Rc/1e3,2)} kHz | Cc = {round(Cc/1e-12,2)} pF")

gm[1] = 16.3363 uS | gm[4] = 163.3628 uS
Rc = 2.35 kHz | Cc = 2.6 pF


In [50]:
# since we know gm12 and the gmid we can calculate the bias current
id[1] = gm[1] / gm_id[1]
id[2] = id[1]
id[0] = 2*id[1]
id[3] = id[0]
id[4] = gm[4] / gm_id[4]
id[5] = id[4]
id[6] = id[3]
i_total = id[3] + 2*id[0] + id[5]
print('i_total (exact) =', round(i_total/1e-6, 1), 'µA')
# we round to 0.5µA bias currents
i_total = max(round(i_total / 1e-6 * 2) / 2 * 1e-6, 0.5e-6)

print(f"id = {id/1e-6} uA")
print('i_total (rounded) =', i_total/1e-6, 'µA')
if i_total < i_total_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!') 

i_total (exact) = 17.4 µA
id = [ 2.17817091  1.08908545  1.08908545  2.17817091 10.89085453 10.89085453
  2.17817091] uA
i_total (rounded) = 17.5 µA
[info] power consumption target is NOT met!


In [51]:
# we calculate the dc gain
gm_gds[1] = n.lookup('GM_GDS', GM_ID=gm_id[1], L=l[1], VDS=0.3, VSB=0)
gm_gds[2] = p.lookup('GM_GDS', GM_ID=gm_id[2], L=l[2], VDS=0.3, VSB=0)
gm_gds[4] = p.lookup('GM_GDS', GM_ID=gm_id[4], L=l[4], VDS=0.3, VSB=0)
gm_gds[5] = n.lookup('GM_GDS', GM_ID=gm_id[5], L=l[5], VDS=0.3, VSB=0)

gds[1] = gm[1] / gm_gds[1]
gm[2] = gm_id[2] * id[2]
gds[2] = gm[2] / gm_gds[2]

gds[4] = gm[4] / gm_gds[4]
gm[5] = gm_id[5] * id[5]
gds[5] = gm[5] / gm_gds[5]

a0 = gm[1] / (gds[1] + gds[2]) * gm[4] / (gds[4] + gds[5])
print('a0 =', round(20*np.log10(a0), 1), 'dB')

a0 = 83.8 dB


In [ ]:
# Parasitic caps at 1st stage output (Miller node)
gm_cgd_m12 = n.lookup('GM_CGD', GM_ID=gm_id_m12, L=l_12, VDS=0.9, VSB=0)
gm_cdd_m12 = n.lookup('GM_CDD', GM_ID=gm_id_m12, L=l_12, VDS=0.9, VSB=0)
gm_cgd_m7  = p.lookup('GM_CGD', GM_ID=gm_id_m7,  L=l_7,  VDS=0.9, VSB=0)

C1 = abs(gm_m12/gm_cgd_m12) + abs(gm_m12/gm_cdd_m12) + abs(gm_m7/gm_cgd_m7)

# Parasitics at output node (2nd stage output)
gm_cdd_m7 = p.lookup('GM_CDD', GM_ID=gm_id_m7, L=l_7, VDS=0.9, VSB=0)
gm_cdd_m8 = n.lookup('GM_CDD', GM_ID=gm_id_m8, L=l_8, VDS=0.9, VSB=0)

C2 = abs(gm_m7/gm_cdd_m7) + abs(gm_m8/gm_cdd_m8)

# Total load including parasitics
C_total_load = c_load + C2
C_total_firstnode = C1  # used with Cc

print("Parasitic @ Miller node (C1):", round(C1/1e-15, 2), "fF")
print("Parasitic @ output node (C2):", round(C2/1e-15, 2), "fF")

# Unity gain frequency approximation including parasitics
f_UGF = gm_m12 / (2*np.pi*(Cc + C1 + C_total_load))

print("Estimated UGF with parasitics =", round(f_UGF/1e6, 2), "MHz")


In [ ]:
# we can now look up the VGS of the MOSFET
vgs_m12 = n.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=0.9, VSB=0.0)
vgs_m34 = p.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=0.9, VSB=0.0) 
vgs_m56 = n.look_upVGS(GM_ID=gm_id_m56, L=l_56, VDS=0.9, VSB=0.0) 
vgs_m7 = p.look_upVGS(GM_ID=gm_id_m7, L=l_7, VDS=0.9, VSB=0.0) 
vgs_m8 = n.look_upVGS(GM_ID=gm_id_m8, L=l_8, VDS=0.2, VSB=0.0) 

print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')
print('vgs_56 =', round(float(vgs_m56), 3), 'V')
print('vgs_7 =', round(float(vgs_m7), 3), 'V')
print('vgs_8 =', round(float(vgs_m8), 3), 'V')

In [ ]:
# calculate settling time due to slewing with the calculated bias current
t_slew = (c_load + c_load_parasitic) * output_voltage / i_total
print('slewing time =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw)
print('settling time =', round(t_settle/1e-6, 3), 'µs')

In [ ]:
# calculate voltage gain error
gain_error = a0 / (1 + a0)
print('voltage gain error =', round((gain_error-1)*100, 1), '%')

In [ ]:
# calculate total rms output noise
sth_m12 = n.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=0.75, VSB=0) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)

sth_m34 = p.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=0.75, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)

output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34 * gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')

In [52]:
# calculate all widths
id_w = np.zeros(len(gm_id))
w = np.zeros(len(gm_id))
w_round = np.zeros(len(gm_id))

id_w[1] = n.lookup('ID_W', GM_ID=gm_id[1], L=l[1], VDS=0.3, VSB=0)
w[1] = id[1]/ id_w[1]
w_round[1] = max(round(w[1]*2)/2, 0.5)

id_w[2] = p.lookup('ID_W', GM_ID=gm_id[2], L=l[2], VDS=0.3, VSB=0)
w[2] = id[2] / id_w[2]
w_round[2] = max(round(w[2]*2)/2, 0.5) 

id_w[0] = n.lookup('ID_W', GM_ID=gm_id[0], L=l[0], VDS=0.3, VSB=0)
w[0] = id[0] / id_w[0]
w_round[0] = max(round(w[0]*2)/2, 0.5)

w[3] = 4 * w_round[0]
# id_w[3] = n.lookup('ID_W', GM_ID=gm_id[3], L=l[3], VDS=0.3, VSB=0)
# w[3] = id[3] / id_w[3]
w_round[3] = max(round(w[3]*2)/2, 0.5)

id_w[4] = p.lookup('ID_W', GM_ID=gm_id[4], L=l[4], VDS=0.3, VSB=0)
w[4] = id[4] / id_w[4]
w_round[4] = max(round(w[4]*2)/2, 0.5) 

# w[5] = w[0] / id[0] * id[5]
id_w[5] = n.lookup('ID_W', GM_ID=gm_id[5], L=l[5], VDS=0.3, VSB=0)
w[5] = id[5] / id_w[5]
w_round[5] = max(round(w[5]*2)/2, 0.5)

id_w[6] = p.lookup('ID_W', GM_ID=gm_id[6], L=l[6], VDS=0.3, VSB=0)
w[6] = id[6] / id_w[6]
w_round[6] = max(round(w[0]*2)/2, 0.5)

gm[0] = gm_id[0] * id[0]
Rs = 1/gm[0] / 1e3
print(Rs)
print(w)
print(w_round)

91.82015947609345
[  0.76908386   3.28807815   8.32813616   4.         203.47168902
  18.04161479   2.61889443]
[  1.    3.5   8.5   4.  203.5  18.    1. ]


In [ ]:
vout_max = 1.7
vout_min = 0.2

# Headroom checks
headroom_M1 = vdd_min - vgs_m34 + vgs_m12 - vin_max
headroom_M4 = vdd_min - vin_max
headroom_M5 = vin_min - vgs_m12
headroom_M7 = vout_max - vgs_m7
headroom_M8 = vdd_min - vout_min - vgs_m8

print(f"Headroom M1 = {headroom_M1:.3f} V")
print(f"Headroom M4 = {headroom_M4:.3f} V")
print(f"Headroom M5 = {headroom_M5:.3f} V")
print(f"Headroom M7 = {headroom_M7:.3f} V")
print(f"Headroom M8 = {headroom_M8:.3f} V")

In [ ]:
# print out final design values
print('5T-OTA dimensioning:')
print('--------------------')
print('M1/2 W=', w_12_round, ', L=', l_12)
print('M3/4 W=', w_34_round, ', L=', l_34)
print('M5   W=', w_5_round, ', L=', l_56)
print('M6   W=', w_6_round, ', L=', l_56)
print()
print('5T-OTA performance summary:')
print('---------------------------')
print('supply current =', round(i_total/1e-6, 1), 'µA')
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')
print('voltage gain error =', round((gain_error-1)*100, 1), '%')
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')
print('turn-on time (slewing+settling) =', round((t_slew+t_settle)/1e-6, 3), 'µs')
print()
print('5T-OTA bias point check:')
print('------------------------')
print('headroom M1 =', round(vdd_min-vgs_m34+vgs_m12-vin_max, 3), 'V')
print('headroom M4 =', round(vdd_min-vin_max, 3), 'V')
print('headroom M5 =', round(vin_min-vgs_m12, 3), 'V')